In [1]:
from dotenv import load_dotenv

load_dotenv()


True

## Setup Tools


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

class MCPService:
    def __init__(self, config: dict):
        # transport "http" is the 2025 standard for remote servers
        self.client = MultiServerMCPClient(config)
        self.tools = []

    async def get_tools(self):
        # In 2025, fetch tools from the client directly
        # Ensure your client stays 'entered' via a context manager in the main loop
        self.tools = await self.client.get_tools()
        return self.tools

In [3]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

## Create State

In [5]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## Create Subagents


In [15]:
# 1. Define the function
def initialize_travel_agent(tools):
    return create_agent(
        model="gpt-5-nano",
        tools=tools,
        system_prompt="""
        You are a travel agent. Search for flights to the desired destination wedding location.
        You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
        - Price (lowest, economy class)
        - Duration (shortest)
        - Date (time of year which you believe is best for a wedding at this location)
        To make things easy, only look for one ticket, one way.
        You may need to make multiple searches to iteratively find the best options.
        You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
        Once you have found the best options, let the user know your shortlist of options."""
    )

In [16]:
from langchain.agents import create_agent

# Venue agent
venue_agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [17]:
# Playlist agent
playlist_agent = create_agent(
    model="gpt-5-nano",
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

## Main Coordinator


In [18]:
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.types import Command  # <--- FIX: Add this line
from langgraph.prebuilt import InjectedState

@tool
async def search_flights(origin: str, destination: str) -> str:
    """
    Travel agent searches for flights to the desired destination wedding location.
    Args:
        origin: The city the guests are departing from (e.g., 'London')
        destination: The wedding destination city (e.g., 'Paris')
    """
    print(f"DEBUG: Searching flights from {origin} to {destination}...")
    # Simply call the sub-agent with the strings provided by the coordinator
    response = await travel_agent.ainvoke({
        "messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]
    })
    
    print(f"DEBUG: Search complete. Found {len(response['messages'])} messages.")
    # Return the content of the last message
    return response['messages'][-1].content
    
@tool
def search_venues(destination: str, guest_count: int) -> str:
    """
    Search for wedding venues based on location and capacity.
    Args:
        destination: The city/location for the wedding.
        guest_count: The total number of guests.
    """
    # No more runtime.state lookup needed!
    query = f"Find wedding venues in {destination} for {guest_count} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(genre: str) -> str:
    """
    Playlist agent curates the perfect playlist for the given genre.
    Args:
        genre: The musical genre (e.g., 'jazz', 'rock').
    """
    # No more runtime.state lookup
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(
    origin: str, 
    destination: str, 
    guest_count: int, 
    genre: str, 
    # This ID is automatically stripped from the LLM's view and injected by the runtime
    tool_call_id: Annotated[str, InjectedToolCallId] 
) -> Command:
    """Update the wedding plan state when all details (origin, destination, guest count, genre) are known."""
    return Command(
        update={
            "origin": origin,
            "destination": destination,
            "guest_count": guest_count,
            "genre": genre,
            # tool_call_id is required to link this message back to the LLM's request
            "messages": [ToolMessage("Wedding state successfully updated.", tool_call_id=tool_call_id)]
        }
    )


In [19]:
async def run_wedding_orchestrator(user_input):
    global travel_agent
    config = {"travel_server": {"transport": "http", "url": "https://mcp.kiwi.com"}}
    # 1. Initialize client normally (NO 'async with')
    client = MultiServerMCPClient(config)
    
    # 2. Fetch tools directly
    mcp_tools = await client.get_tools()
    
    # 3. Initialize agents as before
    travel_agent = initialize_travel_agent(mcp_tools)
        
    # Now, tools like search_flights will have an active connection
    coordinator = create_agent(
        model="gpt-5-nano",
        tools=[search_flights, search_venues, suggest_playlist, update_state],
        state_schema=WeddingState,
        system_prompt="""
    You are a wedding coordinator. 
    1. Immediately call 'search_flights' and 'search_venues'. 
    2. Do not plan anything until you have results from these tools.
    3. First find all the information you need to update the state. Once that is done you can delegate the tasks.
    4. Once you have received their answers, coordinate the perfect wedding for me.
    """
    )

     # 4. Invoke the agent. It will now handle HTTP requests per tool call.
    return await coordinator.ainvoke({"messages": [HumanMessage(content=user_input)]})



## Test


In [20]:
from langchain.messages import HumanMessage

response = await run_wedding_orchestrator(
    "I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre"
)

# async for chunk in coordinator.astream(
#     {"messages": [HumanMessage(content="Your request here")]},
#     stream_mode="updates"
# ):
#     for node_name, state_update in chunk.items():
#         print(f"--- Node: {node_name} ---")
#         if "messages" in state_update:
#             print(state_update["messages"][-1].content)

DEBUG: Searching flights from London to Paris...
DEBUG: Search complete. Found 7 messages.


In [21]:
from pprint import pprint

pprint(response)

{'destination': 'Paris',
 'genre': 'jazz',
 'guest_count': 100,
 'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre", additional_kwargs={}, response_metadata={}, id='feb313a8-6233-4b8d-a0f5-e6bae933c424'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 512, 'prompt_tokens': 416, 'total_tokens': 928, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsIQZjrbshwg1TW5gmUrWlpgidfUo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6cc7-ab43-7053-980a-0d57f25eaf0a-0', tool_calls=[{'name': 'search_flights', 'args': {'origin': 'London', 'destinat

In [22]:
print(response["messages"][-1].content)

Fantastic! I’ve started lining up core pieces for a Paris wedding for 100 guests with a jazz vibe. Here’s what I found so far and the next steps to tighten things up.

What I found for you

Flight options (London → Paris)
- Cheapest options (price-focused)
  - Option A: SEN → CDG
    - When: 18 Oct 2026
    - Price: €29
    - Duration: ~1h10m
  - Option B: SEN → CDG
    - When: 15 Oct 2026
    - Price: €33
    - Duration: ~1h50m
- Shortest durations (time-focused)
  - Option X: Luton → CDG
    - When: 1 Jun 2026
    - Price: €49
    - Duration: ~1h20m
  - Option Y: Luton → CDG
    - When: 2 Jun 2026
    - Price: €49
    - Duration: ~1h20m

Notes
- The results show dates in 2026 and include multiple London-area airports (SEN, LTN). If you have a preferred travel window (e.g., early June 2026 for nicer weather and reasonable fares), I can refine to that window and check availability for your group.

Wedding venues in Paris (capacity ~100)
- Le Cabanon (Paris 9e)
  - Capacity: Private pri

link to trace: https://smith.langchain.com/public/7b5fe668-d3e3-4af4-b513-a8cacc0c9e84/r